First, establish an ssh tunnel [with your PNI password] to jtb3-dev.princeton.edu to make the database available to you:

    ssh username@jtb3-dev.princeton.edu -L 3306:127.0.0.1:3306 -N

jtb3-dev is the database server on which all behavior and mouse data is stored. Second, make sure you have installed datajoint >= '0.12.dev3'
[ pip install datajoint==0.12.dev3 ]

In [1]:
import pylab as pl
import datajoint as dj
import numpy as np
import os
from flask_login import UserMixin
assert dj.__version__=='0.12.dev3'

In [22]:
schema = dj.schema('ahoag_lightsheet_demo')

In [3]:
# schema=dj.create_virtual_module(module_name='lightsheet',schema_name='ahoag_lightsheet_demo')

In [56]:
@schema
class User(dj.Lookup,UserMixin):
    definition = """
    # Users of the light sheet microscope
    username : varchar(20)      # user in the lab
    ---
    email       : varchar(50)
    """
    

@schema
class Experiment(dj.Manual):
    definition = """ # Experiments performed using the light sheet microscope
    experiment_id           :   smallint auto_increment    # allowed here are sql datatypes.
    ----
    username                :   varchar(20)
    title                   :   varchar(100)
    description             :   varchar(250)
    species                 :   varchar(50)
    clearing_protocol       :   enum("iDISCO+_immuno","iDISCO abbreviated clearing","uDISCO","iDISCO+","iDISCO_EdU")
    fluorophores            :   varchar(100)
    primary_antibody        :   varchar(100)
    secondary_antibody      :   varchar(100)
    image_resolution        :   enum("1.3x","4x")
    cell_detection          :   tinyint
    registration            :   tinyint
    probe_detection         :   tinyint
    injection_detection     :   tinyint
    """

In [57]:
User()

username user in the lab,email
ahoag,ahoag@princeton.edu


In [58]:
Experiment() #Shows the current content of the database

experiment_id allowed here are sql datatypes.,username,title,description,species,clearing_protocol,fluorophores,primary_antibody,secondary_antibody,image_resolution,cell_detection,registration,probe_detection,injection_detection


## 2) Generate content for the database

In [59]:
home_dir = os.environ['HOME']
dj_dir = home_dir + '/Documents/Datajoint/'
csv_file = dj_dir + '/pni_core_facility_request_responses.txt'
import csv

In [60]:
def fill_user_table():
    with open(csv_file, mode='r') as infile:
        reader = csv.reader(infile)
        data_dict = {}
        next(reader) # skips the header
        index = 0
        for row in reader:
            email = row[0]
            username = email.split('@')[0].lower()
            insert_user_list = [username,email]
#             print(insert_user_list)
            User().insert((insert_user_list,),skip_duplicates = True)
            index+=1

column_names = ['username','title','description','species','clearing_protocol',
                'fluorophores','primary_antibody','secondary_antibody',
                'image_resolution','cell_detection','registration',
                'probe_detection','injection_detection']
def fill_exp_table():
    with open(csv_file, mode='r') as infile:
        reader = csv.reader(infile)
        data_dict = {}
        next(reader) # skips the header
    #     rows_2insert = [row[0]]
        index = 0
        for row in reader:
            email = row[0]
            username = email.split('@')[0].lower()
#             insert_exp_list = [username,index] + row[2:]  
            insert_exp_list = [username] + row[2:]
            insert_dict = {column_names[ii]:insert_exp_list[ii] for ii in range(len(column_names))}
            Experiment().insert1(insert_dict,skip_duplicates=True)
            index+=1
    #     response_dict = {rows[0]:rows[1] for rows in reader}
    # print(keys)

In [61]:
fill_user_table()

In [62]:
User()

username user in the lab,email
ahoag,ahoag@princeton.edu


In [63]:
fill_exp_table()

In [65]:
Experiment()

experiment_id allowed here are sql datatypes.,username,title,description,species,clearing_protocol,fluorophores,primary_antibody,secondary_antibody,image_resolution,cell_detection,registration,probe_detection,injection_detection
1,ahoag,20190507_Wanglab_exp,Image c-fos in whole brains at 1.3x.,mouse,iDISCO+,None,Anti cfos 1:1000,Donkey antirabbit AF647,1.3x,1,0,0,0
2,ahoag,20190304_LW_m340,Detect placement of optical fiber tip,mouse,iDISCO abbreviated clearing,GCampxDat-Cre Double Transgentic Mouse,None,None,1.3x,0,0,1,0
3,ahoag,201905_atlas00x where x=1:n,"Brody lab rat atlas, iDisco + 1.3x",rat,iDISCO abbreviated clearing,None,None,None,1.3x,0,0,0,0
4,ahoag,an1-31,Recover mcherry expression in cerebellum. Morphology analysis.,mouse,iDISCO+,mcherry,rabbit anti-RFP 1:1000,Donkey antiRabbit 647 1:450,1.3x,0,0,0,1
5,ahoag,Test experiment,A new test experiment,primate,uDISCO,None at all,A very expensive new antibody,,1.3x,1,0,1,0


In [72]:
Experiment().fetch("KEY")[-1]['experiment_id']

5